<a href="https://colab.research.google.com/github/hopperrr/segment_geospatial/blob/main/image_collection_alpha_transarency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This takes a map in PNG format (with associated "PGW" file) and creates multiple segmented images.
#Put the PNG and PGW files in the '/content/images/' folder after the cell following this one has been run.

In [1]:
!git clone https://github.com/facebookresearch/segment-anything.git
!pip install rasterio
!mkdir images

Cloning into 'segment-anything'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 279 (delta 92), reused 83 (delta 80), pack-reused 136
Receiving objects: 100% (279/279), 18.38 MiB | 14.76 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 36.0 MB/s eta 0:00:00


# Automatically generating object masks with SAM

In [2]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
    
    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
        
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

## Set-up

In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from pathlib import Path
imagepath = '/content/images/'
imagename = 'subject.png'
imagename_no_extension = imagename.rsplit( ".", 1 )[ 0 ]
pgwpath = imagepath + imagename_no_extension + ".pgw"
outputfullpath = '/content/output/' + imagename
outputsmall = '/content/output/' + imagename_no_extension + '/smalls'
imagefullpath= imagepath + imagename
metapath = '/content/output/' + imagename_no_extension
image = cv2.imread(imagefullpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [5]:
!python /content/segment-anything/scripts/amg.py --model-type "vit_h" --checkpoint /content/sam_vit_h_4b8939.pth --input {imagefullpath} --output /content/output

Loading model...
Processing '/content/images/subject.png'...
Done!


In [6]:
df = pd.read_csv(metapath +"/metadata.csv")
pg = pd.read_csv(pgwpath, header=None)
image = cv2.imread(imagefullpath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
filecount = len(df)
Path(outputsmall).mkdir(parents=True, exist_ok=True) 
for index, row in df.iterrows():
  itemnum = row['id']

  masker = cv2.imread(metapath + '/' + str(int(itemnum)) + '.png')
  masker_result = cv2.bitwise_and(image, masker)
  details = df.loc[df['id'] == itemnum]
  x = int(details.iloc[0]['bbox_x0'])
  y = int(details.iloc[0]['bbox_y0'])
  x1 = int(x + details.iloc[0]['bbox_w'])
  y1 = int(y + details.iloc[0]['bbox_h'])
  cropped = masker_result[y:y1, x:x1]
  alpha = np.sum(cropped, axis=-1) > 0
  alpha = np.uint8(alpha * 255)
  res = np.dstack((cropped, alpha))
  res = cv2.cvtColor(res, cv2.COLOR_BGRA2RGBA)
  cv2.imwrite(outputsmall + '/' + str(int(itemnum)) + '.png', res)
  pgwExport = open(outputsmall + '/' + str(int(itemnum)) + '.pgw',"w")
  pgwCornerX = pg.iloc[4].values[0] + (x * pg.iloc[0].values[0])
  pgwCornerY = pg.iloc[5].values[0] - (y * pg.iloc[0].values[0])
  pgwExport.write(str(pg.iloc[0].values[0]) + "\n")
  pgwExport.write("0\n")
  pgwExport.write("0\n")
  pgwExport.write(str(pg.iloc[3].values[0]) + "\n")
  pgwExport.write(str(pgwCornerX) + "\n")
  pgwExport.write(str(pgwCornerY) + "\n")
  pgwExport.close()

#plt.figure(figsize=(20,20))
#plt.imshow(res)
#plt.axis('off')
#plt.show()

In [ ]:
zipname = imagename_no_extension + ".zip"
!zip -r {zipname} {outputsmall}


In [ ]:
from google.colab import files
files.download(zipname)

In [ ]:
#Optional - just a reality check to see the dimensions
import rasterio
with rasterio.open('/content/images/subject.png') as src:
    print(src.width, src.height)
    print(src.crs)
    print(src.transform)
    print(src.count)
    print(src.indexes)

1978 1333
None
| 0.51, 0.00, 1599057.27|
| 0.00,-0.51, 5064708.65|
| 0.00, 0.00, 1.00|
4
(1, 2, 3, 4)
